In [1]:
#%%writefile ../crypto_chaotic/audioClient.py
#!/usr/bin/env python

import pyaudio
import wave
from twisted.internet.protocol import DatagramProtocol
from twisted.internet import reactor
import numpy as np
from crypto_chaotic.modulation import drawfft

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 1
#WAVE_OUTPUT_FILENAME = "output.wav"

class EchoClientDatagramProtocol(DatagramProtocol):
    p = pyaudio.PyAudio()

    inputStream = p.open(format=FORMAT,
                         channels=CHANNELS,
                         rate=RATE,
                         input=True,
                         frames_per_buffer=CHUNK)

    outputStream = p.open(format=FORMAT,
                          channels=CHANNELS,
                          rate=RATE,
                          output=True)
    framectr = 0
    
    def startProtocol(self):
        self.transport.connect('127.0.0.1', 8000)
        self.sendDatagram()
    
    def sendDatagram(self):
        #print self.framectr
        if self.framectr <= int(RATE / CHUNK * RECORD_SECONDS):#if len(self.strings):
            data = self.inputStream.read(CHUNK)            
            #if self.framectr == 1:
            #    buff = np.frombuffer(datagram, dtype=np.float64)
            #    drawfft(buff)
            #print buff
            self.transport.write(data)
        else:
            self.inputStream.stop_stream()
            self.inputStream.close()
            self.outputStream.stop_stream()
            self.outputStream.close()
            self.p.terminate()
            reactor.stop()
        self.framectr += 1

    def datagramReceived(self, datagram, host):
        #print 'Datagram received: ', repr(datagram)
        if self.outputStream:
            self.outputStream.write(datagram)
        self.sendDatagram()

def main():
    protocol = EchoClientDatagramProtocol()
    t = reactor.listenUDP(0, protocol)
    reactor.run()

if __name__ == '__main__':
    main()

/home/abhishek/usr/local/lib/python2.7/site-packages/cffi/model.py:526: UserWarning: 'point_conversion_form_t' has no values explicitly defined; next version will refuse to guess which integer type it is meant to be (unsigned/signed, int/long)
  % self._get_c_name())


In [ ]:
%%bash
pushd ../crypto_chaotic
/home/abhishek/usr/local/bin/python -m audioClient

In [ ]:
import pyaudio